In [1]:
import nltk
import threading
import queue
import feedparser
import uuid

In [2]:
threads = []
queues = [queue.Queue(), queue.Queue()]

In [3]:
def extractWords():
    url = 'https://timesofindia.indiatimes.com/rssfeeds/1081479906.cms'
    feed = feedparser.parse(url)
    for entry in feed['entries'][:5]:
        text = entry['title']
        if 'ex' in text:
            continue
        words = nltk.word_tokenize(text)
        data = {'uuid': uuid.uuid4(), 'input': words}
        queues[0].put(data, True)
        print(">> {} : {}".format(data['uuid'], text))

In [4]:
def extractPOS():
    while True:
        if queues[0].empty():
            break
        else:
            data = queues[0].get()
            words = data['input']
            postags = nltk.pos_tag(words)
            queues[0].task_done()
            queues[1].put({'uuid': data['uuid'], 'input': postags}, True)

In [5]:
def extractNE():
    while True:
        if queues[1].empty():
            break
        else:
            data = queues[1].get()
            postags = data['input']
            queues[1].task_done()
            chunks = nltk.ne_chunk(postags, binary=False)
            print("  << {} : ".format(data['uuid']), end = '')
            for path in chunks:
                try:
                    label = path.label()
                    print(path, end=', ')
                except:
                    pass
            print()

In [6]:
def runProgram():
    e = threading.Thread(target=extractWords())
    e.start()
    threads.append(e)

    p = threading.Thread(target=extractPOS())
    p.start()
    threads.append(p)

    n = threading.Thread(target=extractNE())
    n.start()
    threads.append(n)

    queues[0].join()
    queues[1].join()

    for t in threads:
        t.join()

In [7]:
if __name__ == '__main__':
    runProgram()

>> 53561e78-3d5a-47aa-9f6e-dee2d7b3cfb8 : Krishna Shroff on Taapsee being body shamed
>> 12f58e09-ea74-4832-8e87-fa3046b98b48 : Warda mourns the loss of Divya Bharti’s father
>> f0cd79b3-cc57-4c8a-8d6c-c6d157c3ad56 : Akshay is all praise for Bear Grylls
>> 3e6b7262-8ec9-44f3-88ee-20124ef3d644 : 5 iconic moments of SRK's films
>> e8b1cfdf-bd17-480e-87b9-b18833a155ea : Maheep shares rare pics of Shanaya
  << 53561e78-3d5a-47aa-9f6e-dee2d7b3cfb8 : (PERSON Krishna/NNP), (ORGANIZATION Shroff/NNP), 
  << 12f58e09-ea74-4832-8e87-fa3046b98b48 : (GPE Warda/NNP), (PERSON Divya/NNP Bharti/NNP), 
  << f0cd79b3-cc57-4c8a-8d6c-c6d157c3ad56 : (GPE Akshay/NNP), (PERSON Bear/NNP Grylls/NNP), 
  << 3e6b7262-8ec9-44f3-88ee-20124ef3d644 : (ORGANIZATION SRK/NNP), 
  << e8b1cfdf-bd17-480e-87b9-b18833a155ea : (GPE Maheep/JJ), (GPE Shanaya/NNP), 
